# Bayes Stuff

In [1]:
# (Uncomment and) Run this cell to properly import from another Jupyter Notebook (in this case my functions notebook)
# !pip install ipynb

In [2]:
from ipynb.fs.full.functions import *

In [3]:
df = pd.read_csv('../data/clean_data.csv')
df.shape

(9731, 15)

In [4]:
df['subreddit_binary'] = np.where(df['subreddit'] == 'AMA', 0, 1)
df.drop(columns='subreddit')
df.head(2)

,Unnamed: 0,author,author_fullname,author_premium,body,id,is_submitter,link_id,no_follow,parent_id,score,send_replies,subreddit,total_awards_received,post_length,subreddit_binary
0,0,AgronLovesSteel,t2_68gsl4ib,False,Omg sooo good just finished! Gotta have some f...,gvrcgqn,True,t3_mxukth,True,t1_gvrccgy,1,True,AMA,0,68,0
1,1,Adventurous-Mess9304,t2_8x6i6s4f,False,Great answer,gvrcf1t,False,t3_mxnfsk,True,t1_gvqnjl0,2,True,AMA,0,12,0


In [5]:
# df = df.sample(2000)

In [7]:
# X = df.drop(columns=['subreddit_binary', 'subreddit']) #['body']
X = df['body']
y = df['subreddit_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=3)

In [8]:
(X_train.shape, y_train.shape)

((7298,), (7298,))

In [9]:
(X_test.shape, y_test.shape)

((2433,), (2433,))

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

In [11]:
from sklearn.experimental import enable_hist_gradient_boosting # New LigthGBM - must be enabled to use!!
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor


In [72]:
pipe = make_pipeline(CountVectorizer(), TfidfTransformer(), AdaBoostClassifier())#RandomForestClassifier())# MultinomialNB())
params = { 
    'countvectorizer__stop_words': [ 'english', None ],
    'countvectorizer__ngram_range': [ (1,1), (1,2), (2,2), (2,3), (3,3) ],
    'countvectorizer__max_features': [ 500, 10000, 2000, 5000 ]
}

gs = GridSearchCV(pipe, params, n_jobs=-1, verbose=2)
gs.fit(X_train, y_train)

print(f'TfidfVectorizer/Bayes Train: {gs.score(X_train, y_train)}')
print(f'TfidfVectorizer/Bayes Test:  {gs.score(X_test, y_test)}')
print(f'TfidfVectorizer/Bayes Best Params:\n{gs.best_params_}')
print(f'TfidfVectorizer/Bayes Best Estimator:\n{gs.best_estimator_}')

Fitting 5 folds for each of 40 candidates, totalling 200 fits
TfidfVectorizer/Bayes Train: 0.6971773088517402
TfidfVectorizer/Bayes Test:  0.6921496095355528
TfidfVectorizer/Bayes Best Params:
{'countvectorizer__max_features': 10000, 'countvectorizer__ngram_range': (1, 2), 'countvectorizer__stop_words': None}
TfidfVectorizer/Bayes Best Estimator:
Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=10000, ngram_range=(1, 2))),
                ('tfidftransformer', TfidfTransformer()),
                ('adaboostclassifier', AdaBoostClassifier())])


In [73]:
pd.DataFrame(gs.cv_results_).sort_values('rank_test_score').T

,13,11,21,23,31,33,3,1,37,35,...,16,14,36,34,26,24,38,8,28,18
mean_fit_time,2.144351,1.329804,1.170883,1.781231,1.232568,1.988017,1.442339,1.031194,1.860856,1.165209,...,1.129398,0.781,1.074111,0.749999,0.984599,0.67287,0.709657,0.587414,0.660982,0.770802
std_fit_time,0.095873,0.061012,0.087073,0.026398,0.05553,0.05448,0.021691,0.115403,0.041039,0.043584,...,0.069358,0.035309,0.036211,0.041245,0.019581,0.026754,0.056804,0.029188,0.035531,0.065611
mean_score_time,0.134191,0.082631,0.08654,0.129001,0.087602,0.132381,0.114167,0.083599,0.114363,0.107594,...,0.106199,0.08164,0.100791,0.084,0.096801,0.079428,0.081621,0.071613,0.079001,0.07769
std_score_time,0.020672,0.002468,0.008846,0.006841,0.003774,0.013051,0.005401,0.012012,0.013179,0.011477,...,0.00913,0.010874,0.003906,0.004002,0.004579,0.002362,0.011865,0.006616,0.01358,0.012886
param_countvectorizer__max_features,10000,10000,2000,2000,5000,5000,500,500,5000,5000,...,10000,10000,5000,5000,2000,2000,5000,500,2000,10000
param_countvectorizer__ngram_range,"(1, 2)","(1, 1)","(1, 1)","(1, 2)","(1, 1)","(1, 2)","(1, 2)","(1, 1)","(2, 3)","(2, 2)",...,"(2, 3)","(2, 2)","(2, 3)","(2, 2)","(2, 3)","(2, 2)","(3, 3)","(3, 3)","(3, 3)","(3, 3)"
param_countvectorizer__stop_words,None,None,None,None,None,None,None,None,None,None,...,english,english,english,english,english,english,english,english,english,english
params,"{'countvectorizer__max_features': 10000, 'coun...","{'countvectorizer__max_features': 10000, 'coun...","{'countvectorizer__max_features': 2000, 'count...","{'countvectorizer__max_features': 2000, 'count...","{'countvectorizer__max_features': 5000, 'count...","{'countvectorizer__max_features': 5000, 'count...","{'countvectorizer__max_features': 500, 'countv...","{'countvectorizer__max_features': 500, 'countv...","{'countvectorizer__max_features': 5000, 'count...","{'countvectorizer__max_features': 5000, 'count...",...,"{'countvectorizer__max_features': 10000, 'coun...","{'countvectorizer__max_features': 10000, 'coun...","{'countvectorizer__max_features': 5000, 'count...","{'countvectorizer__max_features': 5000, 'count...","{'countvectorizer__max_features': 2000, 'count...","{'countvectorizer__max_features': 2000, 'count...","{'countvectorizer__max_features': 5000, 'count...","{'countvectorizer__max_features': 500, 'countv...","{'countvectorizer__max_features': 2000, 'count...","{'countvectorizer__max_features': 10000, 'coun..."
split0_test_score,0.682192,0.675342,0.673288,0.667123,0.676712,0.671918,0.668493,0.654795,0.623288,0.627397,...,0.515068,0.519178,0.517808,0.514384,0.516438,0.516438,0.508219,0.510274,0.508904,0.507534
split1_test_score,0.664384,0.661644,0.662329,0.660274,0.646575,0.654795,0.65411,0.64863,0.632877,0.632192,...,0.518493,0.517123,0.518493,0.517808,0.515753,0.517123,0.507534,0.508219,0.508219,0.507534


In [74]:
gs.fit(X, y)
print(f'TfidfVectorizer/Bayes Fit X,y: {gs.score(X, y)}')
print(f'TfidfVectorizer/Bayes Best Params:\n{gs.best_params_}')
print(f'TfidfVectorizer/Bayes Best Estimator:\n{gs.best_estimator_}')


Fitting 5 folds for each of 40 candidates, totalling 200 fits
TfidfVectorizer/Bayes Fit X,y: 0.696742369746172
TfidfVectorizer/Bayes Best Params:
{'countvectorizer__max_features': 5000, 'countvectorizer__ngram_range': (1, 1), 'countvectorizer__stop_words': None}
TfidfVectorizer/Bayes Best Estimator:
Pipeline(steps=[('countvectorizer', CountVectorizer(max_features=5000)),
                ('tfidftransformer', TfidfTransformer()),
                ('adaboostclassifier', AdaBoostClassifier())])


In [75]:
# read in 2 new csv files and predict

df1 = pd.read_csv('../data/2021-04-27_1812_AMA_comments.csv')
df2 = pd.read_csv('../data/2021-04-27_1812_AskReddit_comments.csv')

In [76]:
df_test_pred = pd.concat([df1, df2], axis=0)
df_test_pred.shape

(2000, 37)

In [77]:
df_test_pred = df_test_pred[['author', 'body', 'score', 'subreddit']]

df_test_pred['subreddit_binary'] = np.where(df_test_pred['subreddit'] == 'AMA', 0, 1)
df_test_pred = df_test_pred.sample(n=df_test_pred.shape[0], random_state=3)

df_test_pred.drop_duplicates(inplace=True)

df_test_pred['body'] = np.where(df_test_pred['body'] == '[deleted]', np.nan, df_test_pred['body'])
df_test_pred.dropna(inplace=True)
df_test_pred.head(2)

,author,body,score,subreddit,subreddit_binary
933,katofan1,"Same, not quite as dysfunctional as you but st...",1,AMA,0
459,ausernametoforget,I don't usually check my shower temperature un...,4,AskReddit,1


In [78]:
print(df_test_pred['subreddit_binary'].value_counts())
print(df_test_pred['subreddit_binary'].value_counts(normalize=True))

0    977
1    958
Name: subreddit_binary, dtype: int64
0    0.50491
1    0.49509
Name: subreddit_binary, dtype: float64


In [79]:
X_test_pred = df_test_pred['body']
y_test_pred = df_test_pred['subreddit_binary']

In [80]:
y_preds = gs.predict(X_test_pred)

In [81]:
y_preds = pd.Series(y_preds)
y_preds.name = 'predicted_binary'

In [82]:
gs.score(X_test_pred, y_test_pred)

0.6310077519379845

In [83]:
check_p = pred_df.sample(1)
idx = check_p.index
actual_out = np.where(pred_df.iloc[idx[0]]['subreddit_binary'] == 0, 'AMA', 'AskReddit')
pred_out = np.where(pred_df.iloc[idx[0]]['predicted_binary'] == 0, 'AMA', 'AskReddit')
is_match = np.where(actual_out == pred_out, 'CORRECT!', 'wrong!')
print(f"\n{is_match}")
print(f"Actual:       {idx[0]}    {actual_out}")
print(f"Prediction:   {idx[0]}    {pred_out}\n")



wrong!
Actual:       325    AskReddit
Prediction:   325    AMA



In [108]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import VotingRegressor, VotingClassifier

In [109]:
# ss = StandardScaler()
# X_train_scaled = ss.fit_transform(X_train)
# X_test_scaled = ss.transform(X_test)

In [116]:
vr = VotingClassifier(
#     [('knn', KNeighborsClassifier()), 
    [('ada', AdaBoostClassifier()), 
#      ('lr', LogisticRegression()),
     ('tree', DecisionTreeClassifier())],
    n_jobs=-1,
#     weights=[ 0.4, 0.1, 0.5 ]
    weights=[ 0.4, 0.5 ]
)

In [122]:
X = df[['body']]
y = df['subreddit_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=3)

vr.fit_transform(X_train, y_train)
y_pred_vr = vr.predict(X_test)
y_pred_vr

ValueError: could not convert string to float: 'I celebrate my birthdays. But I remember a girl in the same year as me in secondary school wouldn’t celebrate her birthdays and told me it’s because she’s an Jehovah witness. \n\nhttps://www.google.co.uk/search?q=jehovah+witnesses+not+celebrate+birthdays&amp;client=safari&amp;hl=en-gb&amp;ei=s6OEYJL9B420gQbp-JaABA&amp;oq=Jehovah+witnesses+not&amp;gs_lcp=ChNtb2JpbGUtZ3dzLXdpei1zZXJwEAEYADICCAAyCAgAEBYQChAeMgYIABAWEB4yBggAEBYQHjIGCAAQFhAeMgYIABAWEB4yBggAEBYQHjIGCAAQFhAeOgQIABBHOgQIABBDOgUIABDJA1C1Fli6HGDcJGgAcAF4AIABswGIAeYDkgEDMi4ymAEAoAEByAEIwAEB&amp;sclient=mobile-gws-wiz-serp'